[Trips and User - Leetcode](https://leetcode.com/problems/trips-and-users/description/)

In [ ]:
import findspark
findspark.init()

import datetime

In [ ]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local").appName("playground").getOrCreate()

In [ ]:
trips_schema = T.StructType([
    T.StructField("id", T.IntegerType(), False),
    T.StructField("client_id", T.IntegerType(), False),
    T.StructField("driver_id", T.IntegerType(), False),
    T.StructField("city_id", T.IntegerType(), False),
    T.StructField("status", T.StringType(), False),
    T.StructField("request_at", T.DateType(), False)
    ])

users_schema = T.StructType([
    T.StructField("users_id", T.IntegerType(), False),
    T.StructField("banned", T.StringType(), False),
    T.StructField("role", T.StringType(), False)
])

In [ ]:
trips_data = [
    (1, 1, 10, 1, "completed", datetime.date(2013, 10, 1)),
    (2, 2, 11, 1, "cancelled_by_driver", datetime.date(2013, 10, 1)),
    (3, 3, 12, 6, "completed", datetime.date(2013, 10, 1)),
    (4, 4, 13, 6, "cancelled_by_client", datetime.date(2013, 10, 1)),
    (5, 1, 10, 1, "completed", datetime.date(2013, 10, 2)),
    (6, 2, 11, 6, "completed", datetime.date(2013, 10, 2)),
    (7, 3, 12, 6, "completed", datetime.date(2013, 10, 2)),
    (8, 2, 12, 12, "completed", datetime.date(2013, 10, 3)),
    (9, 3, 10, 12, "completed", datetime.date(2013, 10, 3)),
    (10, 4, 13, 12, "cancelled_by_driver", datetime.date(2013, 10, 3)),
]

trips = spark.createDataFrame(trips_data, trips_schema)

trips.show()

users_data = [
    (1, "No", "client"),
    (2, "Yes", "client"),
    (3, "No", "client"),
    (4, "No", "client"),
    (10, "No", "driver"),
    (11, "No", "driver"),
    (12, "No", "driver"),
    (13, "No", "driver"),
]

users = spark.createDataFrame(users_data, users_schema)

users.show()

In [ ]:
user_trips_client = (trips.alias("t")
                     .join(users.alias("u"), 
                           F.col("t.client_id") == F.col("u.users_id"), 
                           "left"))

user_trips_client_driver = (user_trips_client.alias("utc")
                            .join(users.alias("us"),
                                  F.col("utc.driver_id") == F.col("us.users_id"),
                                  "left"))



user_trips_client_driver.show()

In [ ]:
user_trips_client_driver = user_trips_client_driver.where("(utc.banned = 'No') AND (us.banned = 'No')")

user_trips_client_driver.show()

In [ ]:
user_trips_client_driver = (user_trips_client_driver
                            .withColumn("cancelled_count", 
                                        F.expr("CASE WHEN utc.status = 'completed' THEN 0 ELSE 1 END")))

user_trips_client_driver.show()

In [ ]:
trips_agg_data = (user_trips_client_driver
                  .groupBy("utc.request_at")
                  .agg(F.count("id").alias("completed_count"), 
                       F.sum("cancelled_count").alias("cancelled_count")))

trips_agg_data.show()

In [ ]:
trips_agg_data = trips_agg_data.selectExpr("request_at as Day", 
                                           "round((cancelled_count / completed_count), 2) as cancellation_rate")

trips_agg_data.show()